In [1]:
import wget
import feedparser
import glob
import os
from pydub import AudioSegment
import io
import os
import wave
from google.cloud import storage
from google.cloud import storage
from google.cloud import speech_v1 as speech
from google.oauth2 import service_account
from collections import defaultdict
import numpy as np
import nltk
import psutil
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

### Load CGP folder contains transcripts. Pick a file to start summarizing

In [2]:
storage_client = storage.Client()
bucket = storage_client.bucket('ee6893')

all_files_list = [file.download_as_string().decode('utf=8') for file in list(bucket.list_blobs(prefix = 'transcript/'))]

index = 0
for file in bucket.list_blobs(prefix = 'transcript/'):
    print(index,' ', file.name.split('/')[1])
    index +=1

0   628: In the Shadow of the City.txt
1   745: Getting Out.txt
2   746: This Is Just Some Songs.txt
3   747: Suitable for Children.txt
4   748: The End of the World as We Know It.txt
5   749: My Bad.txt
6   750: The Ferryman.txt
7   751: Audience of One.txt
8   752: An Invitation to Tea.txt
9   753: What We’ve Got Here is Failure to Communicate.txt
10   A Conversation With Dr.txt
11   Facebook vs.txt
12   Facebook’s Plan to Police the Truth.txt
13   Facts Aren't Enough.txt
14   Fake News: An Origin Story.txt
15   Fate, Domestic Terrorism and the Nomination of Merrick Garland.txt
16   Father and Son, Forced Apart at the Border.txt
17   Fear and Loss: Inside India’s Coronavirus Crisis.txt
18   Fear, Fury and the Coronavirus.txt
19   Filmmaker Edgar Wright On 'Last Night in Soho'.txt
20   Filthy Rich.txt
21   Friday, December 3, 2021.txt
22   Screaming Into The Void.txt
23   Screaming into a Void.txt
24   Secret Friends.txt
25   Senator Claire McCaskill on Losing Missouri and the Politic

In [3]:
# loading a file by index
a_file = all_files_list[1]

In [5]:
def nest_sentences(document):
    nested = []
    sent = []
    length = 0
    
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence)
        if length < 1024:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = [sentence]
            length = len(sentence)

    if sent:
        nested.append(sent)
    return nested



### Load Three Transformers

In [6]:

from transformers import GPT2Tokenizer,GPT2LMHeadModel,BartTokenizer, BartForConditionalGeneration, pipeline
from summarizer import TransformerSummarizer


def generate_summary_gpt(nested_sentences):
    summaries = []
    model=TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
    for nested in nested_sentences:
        text = ''.join(nested)
        GPT_summary= ''.join(model(text, min_length = round(0.3 * len(text.split(' ')))))
        summaries.append(GPT_summary)
    summaries = [sentence for sublist in summaries for sentence in sublist]
    return summaries
nested_file = nest_sentences(a_file)
#generate_summary(nested_file)

def generate_summary_bart(nested_sentences):
    summaries = []

    summarizer = pipeline('summarization', model='facebook/bart-large-cnn', tokenizer='facebook/bart-large-cnn')
    for nested in nested_sentences:
        text = ' '.join(nested)
        output = summarizer(text, min_length = round(0.3 * len(text.split(' '))), max_length = round(0.4 * len(text.split(' '))), do_sample=False)
        summaries.append(output)
    summaries = [sentence for sublist in summaries for sentence in sublist]
    return summaries

def generate_summary_bert(nested_sentences):
    summaries = []
    model = TransformerSummarizer()
    for nested in nested_sentences:
        text = ''.join(nested)
        GPT_summary= ''.join(model(text, max_length = round(0.4 * len(text.split(' '))), min_length = round(0.1 * len(text.split(' ')))))
        summaries.append(GPT_summary)
    summaries = [sentence for sublist in summaries for sentence in sublist]
    return summaries



2021-12-19 06:37:44.219691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-19 06:37:44.219744: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
#### BERT Summary

nested_file = nest_sentences(a_file)

int_result_bert = ''.join(generate_summary_bert(nested_file))

''.join(generate_summary_bert(nest_sentences(int_result_bert)))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.tr

"I'm WBEZ Chicago to This American Life. He's slight big glasses. He is knee-deep in water. Point on Earth, where his voice would be. That's why you have to say Canadian citizen.Can you hear me?Okay, you have for Canadian citizens there. I don't want people who hear this to think that. So she was probably in her forties. So she seemed She seemed.Like, make her think I was on her side. I just like, like Push the door as hard as I could to close it. Bad tension enough. Act three, you can't go your own way.She says, you never take responsibility for anything me up. There's a way out to be listening to me. The Luminous innocence of the light curtains in your stomach. She visited them at home to see how Leona's doing.Kids more kids because I'm out of school. I would just care if they are nice or not."

In [29]:
#### GPT Summary

int_result_gpt = ''.join(generate_summary_gpt(nested_file))

int_result_gpt2 = ''.join(generate_summary_bert(nest_sentences(int_result_gpt)))

''.join(generate_summary_bert(nest_sentences(int_result_gpt)))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.tr

"He had two series of phone calls when she recorded with the man you heard.He saying the crowd is increasing its increasing, I know.May need to get up close to ride patient care.They don't want them to make them afraid of the world.Did you know, my dad does the same thing, the sleeping damsel Swift.There's no way to get here from the choices given to you.They were green sashes in the morning, and got to be door.Kornfeld, Seth Windstone Nelson, Catherine."

In [7]:
# Bart Summary

int_result_bart = ' '.join([i['summary_text'] for i in generate_summary_bart(nested_file)])

int_result_bart2 = ' '.join([i['summary_text'] for i in generate_summary_bart(nest_sentences(int_result_bart))])

' '.join([i['summary_text'] for i in generate_summary_bart(nest_sentences(int_result_bart2))])


'CNN.com\'s John Sutter shares a phone call he had with a man he met in Iraq. The man, Kirk Johnson, was working for the U.S. government in Iraq when things started to fall apart in Afghanistan. Johnson reached out to Sutter because he knew Sutter. "I was too young and unfamiliar to identify what a crazy person seems like, but what I could identify in that moment was that this person was unpredictable," she says. "It had that quality of like, like time slows down. And your kind of your there, but you\'re also watching," she adds. "Nice to show everybody that even a kid can make it right, sometimes," she says. "I heard her, and she ran back towards the "I think someone at the hotel just said this to him, and then they gave her a key," she adds. The woman had been in a psychiatric hospital and escaped that night come to the hotel. Leo is having a harder time finding his place in middle school. "I don\'t know anyone here really and I think it\'s just everything in general with overwhelmin